In [ ]:
!pip install openai transformers datasets

In [ ]:
# https://drive.google.com/file/d/1ra3m0uJ4uuYy0hGlgSerzlUSSAPOP3Rm/view?usp=sharing
# https://drive.google.com/file/d/1T-iiuxRzQiDzdJqdg-dc8Teh85AZtn73/view?usp=sharing

!gdown 1ra3m0uJ4uuYy0hGlgSerzlUSSAPOP3Rm
!gdown 1T-iiuxRzQiDzdJqdg-dc8Teh85AZtn73

Downloading...
From: https://drive.google.com/uc?id=1ra3m0uJ4uuYy0hGlgSerzlUSSAPOP3Rm
To: /content/train_with_new_features.csv
100% 649k/649k [00:00<00:00, 138MB/s]
Downloading...
From: https://drive.google.com/uc?id=1T-iiuxRzQiDzdJqdg-dc8Teh85AZtn73
To: /content/test_with_new_features.csv
100% 281k/281k [00:00<00:00, 105MB/s]


# Rule Based Search & Retrieval

In [ ]:
import pandas as pd
import sqlite3

# Load CSV data into pandas dataframes
test_df = pd.read_csv('test_with_new_features.csv')
train_df = pd.read_csv('train_with_new_features.csv')

# Create an in-memory SQLite database and load data
conn = sqlite3.connect(':memory:')  # In-memory database for quick operations
test_df.to_sql('test_jobs', conn, index=False, if_exists='replace')
train_df.to_sql('train_jobs', conn, index=False, if_exists='replace')

# Function to take natural language prompt and convert it to SQL query
# In real world, this question will have a lot more clauses
# We intentionally tried to make it very rudimentary
def generate_sql_query(prompt):
    # Placeholder Llama interaction: You can replace this with actual Llama API or model integration.
    # The LLM should take the user prompt and return a corresponding SQL query.
    if "senior developer" in prompt.lower():
        sql_query = "SELECT * FROM train_jobs WHERE Job_Role = 'Developer' AND Experience_Level = 'Senior'"
    elif "remote" in prompt.lower():
        sql_query = "SELECT * FROM train_jobs WHERE Remote_Work = 1"
    else:
        sql_query = "SELECT * FROM train_jobs LIMIT 5"  # Default query
    return sql_query

# Display SQL query and execute it
def execute_sql_query(sql_query):
    print(f"Generated SQL Query: {sql_query}")
    results_df = pd.read_sql_query(sql_query, conn)
    return results_df

# Generate natural language response explaining the results
def explain_results_with_llm(results_df):
    # Placeholder Llama interaction: You can replace this with actual Llama API or model integration.
    if results_df.empty:
        explanation = "No results were found for your query."
    else:
        explanation = f"The query returned {len(results_df)} jobs matching your criteria. Some examples include {results_df['Job-Title'].iloc[0]} and {results_df['Job-Title'].iloc[1]}."
    return explanation

# Main function to take a prompt, generate SQL, display results, and explain with Llama
def job_search_wo_llm(prompt):
    # Generate SQL query from the natural language prompt
    sql_query = generate_sql_query(prompt)

    # Execute SQL query and display results
    results_df = execute_sql_query(sql_query)

    # Display the SQL query results
    print("SQL Query Results:")
    print(results_df.head())

    # Explain the results using Llama
    explanation = explain_results_with_llm(results_df)
    print(f"Natural Language Explanation: {explanation}")

# Example usage
prompt = input("Enter your job search query: ")
job_search_wo_llm(prompt)


Enter your job search query: Find Python jobs
Generated SQL Query: SELECT * FROM train_jobs LIMIT 5
SQL Query Results:
                                              Resume  \
0  enthusiastic frontend developer with a strong ...   
1  proficient in machine learning algorithms and ...   
2  frontend developer specializing in crafting in...   
3  experienced mobile app developer with expertis...   
4  proficient in frontend technologies such as ht...   

                           Job-Title                     Location  \
0          senior frontend developer                      finland   
1   senior machine learning engineer      new delhi, delhi, india   
2          senior frontend developer  bengaluru, karnataka, india   
3                          developer                   london, uk   
4  intermediate full stack developer                  calgary, ab   

             Salary                                        Description  \
0        Over $150K  the role is senior frontend positi

# Natural Language Analytics

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import sqlite3
import torch

# Log in to Hugging Face (replace with your token)
login(token="hugging_face_token_with_access_to_Llama_3.1", add_to_git_credential=True)

# Load CSV data into pandas dataframes
test_df = pd.read_csv('test_with_new_features.csv')
train_df = pd.read_csv('train_with_new_features.csv')

# Create an in-memory SQLite database and load data
conn = sqlite3.connect(':memory:')  # In-memory database for quick operations
test_df.to_sql('test_jobs', conn, index=False, if_exists='replace')
train_df.to_sql('train_jobs', conn, index=False, if_exists='replace')

# Load Llama 3.1 8B model from Hugging Face for generating SQL queries and explanations
model_name = "meta-llama/Llama-3.1-8B"

# Detect if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    use_auth_token=True
)

# Ensure pad_token_id is defined
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Enter your job search query: Find Python jobs in New York
Generated SQL Query: SELECT * FROM train_jobs WHERE Job_Title LIKE '%Python%' AND Location LIKE '%New York%';
Error executing query: Execution failed on sql 'SELECT * FROM train_jobs WHERE Job_Title LIKE '%Python%' AND Location LIKE '%New York%';': no such column: Job_Title

SQL Query Results:
Empty DataFrame
Columns: []
Index: []

Natural Language Explanation:
No results were found for your query.


In [ ]:
# Function to generate text using the model
def generate_text(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generation_config = {
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
        "max_new_tokens": max_new_tokens,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }
    outputs = model.generate(**inputs, **generation_config)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    generated_text = generated_text[len(prompt):].strip()
    return generated_text

# Function to generate SQL query from natural language prompt using Llama 3.1 with examples
def generate_sql_query_with_llm(prompt):
    # Get the column names from the database
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(train_jobs);")
    columns_info = cursor.fetchall()
    column_names = [info[1] for info in columns_info]
    columns_str = ", ".join(column_names)

    # Prepare the few-shot examples with correct column names
    examples = [
        {
            "nl": "Find Python jobs in New York.",
            "sql": "SELECT * FROM train_jobs WHERE \"Job-Title\" LIKE '%Python%' AND Location LIKE '%New York%';"
        },
        {
            "nl": "Show me remote data science jobs paying over $120,000.",
            "sql": "SELECT * FROM train_jobs WHERE \"Job-Title\" LIKE '%Data Science%' AND Remote_Work = 1 AND Min_Salary >= 120000;"
        },
        {
            "nl": "Find machine learning engineer positions in California with a salary of at least $150,000.",
            "sql": "SELECT * FROM train_jobs WHERE \"Job-Title\" LIKE '%Machine Learning Engineer%' AND Location LIKE '%California%' AND Min_Salary >= 150000;"
        }
    ]

    # Build the prompt with schema information
    llm_prompt = f"### Table Schema:\nColumns in 'train_jobs' table: {columns_str}\n\n"
    for ex in examples:
        llm_prompt += f"### Instruction:\nConvert the following natural language query to SQL using the table schema provided.\nNatural Language Query: {ex['nl']}\n### SQL Query:\n{ex['sql']}\n\n"

    # Add the user's prompt
    llm_prompt += f"### Instruction:\nConvert the following natural language query to SQL using the table schema provided.\nNatural Language Query: {prompt}\n### SQL Query:\n"

    # Generate SQL query using Llama 3.1 model
    response_text = generate_text(llm_prompt, max_new_tokens=200)

    # Extract SQL query from response
    sql_query = response_text.strip().split("\n")[0].strip()

    return sql_query

# Display SQL query and execute it
def execute_sql_query(sql_query):
    print(f"Generated SQL Query: {sql_query}")
    try:
        results_df = pd.read_sql_query(sql_query, conn)
        return results_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()

# Use Llama 3.1 to generate natural language explanation of SQL results
def explain_results_with_llm(results_df):
    if results_df.empty:
        return "No results were found for your query."

    # Convert results to string format
    results_text = results_df.head().to_string(index=False)

    llm_prompt = f"""
The following are job search results from a database query:

{results_text}

Provide a brief explanation summarizing these job search results in simple terms.
"""

    # Generate explanation using Llama 3.1 model
    explanation = generate_text(llm_prompt, max_new_tokens=200)

    return explanation.strip()

# Main function to take a prompt, generate SQL, display results, and explain with Llama 3.1
def job_search_with_llm(prompt):
    # Generate SQL query from the natural language prompt
    sql_query = generate_sql_query_with_llm(prompt)

    # Execute SQL query and display results
    results_df = execute_sql_query(sql_query)

    # Display the SQL query results
    print("\nSQL Query Results:")
    print(results_df.head())

    # Explain the results using Llama 3.1
    explanation = explain_results_with_llm(results_df)
    print(f"\nNatural Language Explanation:\n{explanation}")

# Example usage
prompt = input("Enter your job search query: ")
job_search_with_llm(prompt)

Enter your job search query: Find Python jobs
Generated SQL Query: SELECT * FROM train_jobs WHERE "Job-Title" LIKE '%Python%';

SQL Query Results:
                                              Resume  \
0  as a python developer i leverage my expertise ...   
1  proficient in python with experience in librar...   
2  experienced python developer with strong backg...   
3  innovative python developer with passion for s...   
4  dynamic python developer with a focus on build...   

                      Job-Title                          Location  \
0  entry level python developer  san francisco, ca, united states   
1  entry level python developer  san francisco, ca, united states   
2  entry level python developer  san francisco, ca, united states   
3  entry level python developer  san francisco, ca, united states   
4  entry level python developer  san francisco, ca, united states   

                  Salary                                        Description  \
0  US$75K–US$150K a ye